<a href="https://colab.research.google.com/github/Zaheer505/Tiny-ML/blob/main/tf_lite_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import h5py
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sys import getsizeof
print("Asdasd")

In [ ]:
# Must check this because tf 1.x is different and has different rules.
print(tf.__version__)

2.6.0


In [ ]:
# getting MNIST Fashion data set
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
# Name of classes in our data set
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

**Exploring the data**

In [ ]:
train_images.shape

(60000, 28, 28)

In [ ]:
len(train_labels)

60000

In [ ]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [ ]:
test_images.shape

(10000, 28, 28)

In [ ]:
len(test_labels)

10000

**Preprocessing**

In [ ]:
# To get values between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

**Model**

In [ ]:
# Neural Network Model
model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(10)
])

In [ ]:
# Optimizer, loss and metric to know accuracy 
model.compile(optimizer='adam',
              loss= SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

**Training**

In [ ]:
# Fitting images to Model
# Epoch = number of times that the learning algorithm will work through the entire training dataset
# More Epochs result in more accuracy but can result in overfitting
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4729 - accuracy: 0.8306
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3595 - accuracy: 0.8676
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3265 - accuracy: 0.8795
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3016 - accuracy: 0.8886
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2855 - accuracy: 0.8934
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2719 - accuracy: 0.8974
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2571 - accuracy: 0.9040
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2464 - accuracy: 0.9064
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2359 - accuracy: 0.9096
Epoch 10/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.227

In [ ]:
 tf_model_name = "tf_model_fashion_mnist.h5"

In [ ]:
# to know what our NN model look like, we can get it's summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                2570      
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#saving the
model.save(tf_model_name)

In [ ]:
print ('Size:', round(os.path.getsize(tf_model_name)/(1024*1024), 3) , 'MB')

Size: 1.589 MB


In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', round(test_acc*100, 2), '%')

313/313 - 0s - loss: 0.3482 - accuracy: 0.8808

Test accuracy: 88.08 %


**TF Lite Model Started**

In [ ]:
tf_lite_model_file_name = "tf_lite_model.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE] # optimizing for size 
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpwri52zf2/assets


INFO:tensorflow:Assets written to: /tmp/tmpwri52zf2/assets


In [ ]:
tflite_model_name = tf_lite_model_file_name
open(tflite_model_name, "wb").write(tflite_model)

139408

In [ ]:
print ('Size:', round(os.path.getsize(tf_lite_model_file_name)/(1024), 3) , 'KB')

Size: 136.141 KB


**Input Shape Check**

In [ ]:
interpreter = tf.lite.Interpreter(model_path = tf_lite_model_file_name)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

**Resizing Shape**

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 28, 28))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
test_images.dtype # checking orignal data type

dtype('float64')

In [ ]:
test_imgs_numpy = np.array(test_images, dtype=np.float32) # convert to new data type

**Prediction/Accuracy Check**

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
tf_prediction_classes = np.argmax(tflite_model_predictions, axis=1)
print (tf_prediction_classes)

Prediction results shape: (10000, 10)
[9 2 1 ... 8 1 5]


In [ ]:
acc = accuracy_score(tf_prediction_classes, test_labels)
print ('Accuracy:', round(acc*100, 4), '%')

Accuracy 87.86 %
